<a href="https://colab.research.google.com/github/Diana13Delgado/Challenge-TelecomX-parte-2/blob/main/Challenge_TelecomX_parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**🛠️ Preparación de los Datos**

##**Extracción del Archivo Tratado**

In [2]:
import pandas as pd

In [3]:
datos = pd.read_csv('/content/df_limpio.csv')
datos.head()

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total,cuentas_diarias
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30,2.186667
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40,1.996667
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,2.463333
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85,3.266667
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40,2.796667


##**Eliminación de Columnas Irrelevantes**

In [22]:
#Eliminar duplicados (si existen)
datos = datos.drop_duplicates()

In [23]:
#Identificar valores nulos
print(datos.isnull().sum())

customerID                   0
Churn                        0
customer.gender              0
customer.SeniorCitizen       0
customer.Partner             0
customer.Dependents          0
customer.tenure              0
phone.PhoneService           0
phone.MultipleLines          0
internet.InternetService     0
internet.OnlineSecurity      0
internet.OnlineBackup        0
internet.DeviceProtection    0
internet.TechSupport         0
internet.StreamingTV         0
internet.StreamingMovies     0
account.Contract             0
account.PaperlessBilling     0
account.PaymentMethod        0
account.Charges.Monthly      0
account.Charges.Total        0
cuentas_diarias              0
dtype: int64


In [24]:
# Rellenar nulos en "account.Charges.Total" con la media
datos["account.Charges.Total"] = datos["account.Charges.Total"].fillna(datos["account.Charges.Total"].mean())

In [25]:
#Eliminar columnas irrelevantes
datos = datos.drop(columns=["customerID"])

In [26]:
#Quitar espacios en blanco en variables tipo texto
datos = datos.applymap(lambda x: x.strip() if isinstance(x, str) else x)

/tmp/ipython-input-793279701.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  datos = datos.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [27]:
#Resetear índice después de limpiar
datos = datos.reset_index(drop=True)

In [28]:
#Revisar que ya no haya nulos
print(datos.isnull().sum())

Churn                        0
customer.gender              0
customer.SeniorCitizen       0
customer.Partner             0
customer.Dependents          0
customer.tenure              0
phone.PhoneService           0
phone.MultipleLines          0
internet.InternetService     0
internet.OnlineSecurity      0
internet.OnlineBackup        0
internet.DeviceProtection    0
internet.TechSupport         0
internet.StreamingTV         0
internet.StreamingMovies     0
account.Contract             0
account.PaperlessBilling     0
account.PaymentMethod        0
account.Charges.Monthly      0
account.Charges.Total        0
cuentas_diarias              0
dtype: int64


##**Encoding**

In [30]:
# Reemplazar "No internet service" por "No" en las siguientes columnas:
for col in [
    "internet.OnlineSecurity",
    "internet.OnlineBackup",
    "internet.DeviceProtection",
    "internet.TechSupport",
    "internet.StreamingTV",
    "internet.StreamingMovies"
]:
    datos[col] = datos[col].replace("No internet service", "No")
    print(f"{col}: {datos[col].unique()}")

internet.OnlineSecurity: ['No' 'Yes']
internet.OnlineBackup: ['Yes' 'No']
internet.DeviceProtection: ['No' 'Yes']
internet.TechSupport: ['Yes' 'No']
internet.StreamingTV: ['Yes' 'No']
internet.StreamingMovies: ['No' 'Yes']


In [31]:
# Aplicar oneHotEncoding a todas las columnas categóricas
datos_encoded = pd.get_dummies(datos, drop_first=True)

In [32]:
# Revisar el nuevo shape del dataframe
print("Shape original:", datos.shape)
print("Shape transformado:", datos_encoded.shape)

Shape original: (7043, 21)
Shape transformado: (7043, 26)


In [33]:
# Ver primeras filas
print(datos_encoded.head())

   customer.SeniorCitizen  customer.tenure  account.Charges.Monthly  \
0                       0                9                     65.6   
1                       0                9                     59.9   
2                       0                4                     73.9   
3                       1               13                     98.0   
4                       1                3                     83.9   

   account.Charges.Total  cuentas_diarias  Churn_Yes  customer.gender_Male  \
0                 593.30         2.186667      False                 False   
1                 542.40         1.996667      False                  True   
2                 280.85         2.463333       True                  True   
3                1237.85         3.266667       True                  True   
4                 267.40         2.796667       True                 False   

   customer.Partner_Yes  customer.Dependents_Yes  phone.PhoneService_Yes  ...  \
0                  True

In [34]:
from sklearn.model_selection import train_test_split

In [36]:
#Variable objetivo (0=No, 1=Yes)
y = datos["Churn"].map({"No": 0, "Yes": 1})
X = datos.drop(columns=["Churn"])

In [37]:
#One-hot encoding
X = pd.get_dummies(X, drop_first=True)

In [38]:
#train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [39]:
print("X shape:", X.shape, "| y shape:", y.shape)
print("Train ->", X_train.shape, y_train.shape, "| Test ->", X_test.shape, y_test.shape)

X shape: (7043, 25) | y shape: (7043,)
Train -> (5634, 25) (5634,) | Test -> (1409, 25) (1409,)
